In [1]:
import torch # torch는 딥러닝을 구성하기 편하도록 페이스북에서 만든 딥러닝 라이브러리, 텐서플로우와 같은 딥러닝 라이브러리이다
import torch.nn as nn  # 신경망 구축을 쉽게 해주는 함수

In [2]:
# GPU 확인 , gpu가 없으면 cpu로 연산, gpu가 있으면 gpu로 연산하겠다는 준비, 따라서 device는 cpu나 gpu 문자열로 바뀐다.
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [3]:
# 4명의 데이터가 있는데, 1명당 2개의 컬럼을 가지고 있다. 그리고 그걸 2차원 리스트로 4명의 데이터를 선언 후
# 토치가 이해할 수 있는 텐서타입으로 바꾼다.
# 이때, 텐서타입은 float
# to(device)는 모델이 gpu에서 연산될 것이기 때문에 to(cuda)를 함으로써 데이터를 gpu에 얹어 놓는다.
 
X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]]).to(device)
Y = torch.FloatTensor([[0], [1], [1], [0]]).to(device)

In [5]:
model = nn.Sequential(
    nn.Linear(2,10, bias=True,), # input_layer = 2, hidden_layer1 = 10
    nn.Sigmoid(),
    nn.Sigmoid(),
    nn.Linear(10,10, bias=True,), # hidden_layer1 = 10, hidden_layer2 = 10
    nn.Sigmoid(),
    nn.Linear(10,10, bias=True,), # hidden_layer2 = 10, hidden_layer3 = 10
    nn.Sigmoid(),
    nn.Linear(10,1, bias=True,), # hidden_layer3 = 10, output_layer = 1
    nn.Sigmoid(),
).to(device)

In [11]:
criterion = torch.nn.MSELoss().to(device) # Mean Square Error
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001) # 옵티마이저인 Stochastic Gradient Descent를 사용.
                                                           # 업데이트 대상은 model 객체가 가지고 있는 w들을 대상으로 업데이트를 할 것이고, learning rate는 0.001로 설정할 것이다.

In [12]:
for epoch in range(50): # 50번의 학습
  optimizer.zero_grad() # 옵티마이저의 기울기를 매번 초기화해서 계산해야하기 때문에 초기화 하는 함수를 선언
  # forward 연산
  hypothesis = model(X) # 모델에 데이터를 넣고, propagation을 진행한 후, 최종 아웃풋을 hypothesis에 담는다.

  # 비용 함수
  cost = criterion(hypothesis, Y) # 예측값과 정답값 사이의 Mean Square Loss를 구함
  cost.backward() # backpropagation이 아니다. 이 부분은 기울기를 구하는 함수
  optimizer.step() # 여기서 비로소 backpropagation을 실행해 모델이 모든 w들을 업데이트 한다.

  # 5의 배수에 해당되는 에포크마다 비용을 출력
  if epoch % 5 == 0 :
    print("에폭:", epoch, cost.item())

에폭: 0 0.25263649225234985
에폭: 5 0.25166529417037964
에폭: 10 0.25093400478363037
에폭: 15 0.2504405379295349
에폭: 20 0.2501578629016876
에폭: 25 0.25003504753112793
에폭: 30 0.25000813603401184
에폭: 35 0.2500186264514923
에폭: 40 0.2500295341014862
에폭: 45 0.2500287890434265


In [14]:
X_2 = torch.FloatTensor([[0,0], [0,1]]).to(device)
hypothesis = model(X_2) # fitting된 모델에 데이터를 넣어서 inference하는 부분
print(hypothesis)

tensor([[0.4963],
        [0.4959]], grad_fn=<SigmoidBackward0>)


In [15]:
#
#
# minibatch DNN
#
#

In [16]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더

In [6]:
# GPU 확인
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [19]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
# 정답값은 1사람당 1개
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [22]:
# X 데이터셋과 Y 데이터 셋을 포장함.
dataset = TensorDataset(x_train, y_train)
print(list(dataset))

[(tensor([73., 80., 75.]), tensor([152.])), (tensor([93., 88., 93.]), tensor([185.])), (tensor([89., 91., 90.]), tensor([180.])), (tensor([ 96.,  98., 100.]), tensor([196.])), (tensor([73., 66., 70.]), tensor([142.]))]


In [23]:
# 2개 데이터를 1세트로 만들어서 전체데이터셋을 나눈다. 이때, n개의 세트들의 순서를 shuffle한다.
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [25]:
model = nn.Sequential(
    nn.Linear(3, 10, bias=True), # input_layer = 3, hidden_layer1=10
    nn.ReLU(),
    nn.Linear(10, 10, bias=True), # hidden_layer1 = 10, hidden_layer2=10
    nn.ReLU(),
    nn.Linear(10, 10, bias=True), # hidden_layer2 = 10, hidden_layer3=10
    nn.ReLU(),
    nn.Linear(10, 1, bias=True), # hidden_layer3 = 10, output_layer=1
    nn.ReLU()
).to(device)

In [26]:
criterion = torch.nn.MSELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5) # 0.00001

In [29]:
# minibatch로 실행할 경우 backpropagation이 더 빈번하게 일어난다.
nb_epochs = 5
for epoch in range(nb_epochs + 1): # 전체 데이터 셋으로 5번 학습할 것이다.
  for batch_idx, samples in enumerate(dataloader): # minibatch 데이터를 반환하면서 index도 반환한다. GPU폭발 방지를 위해 minibatch로 실행
    
    x_train, y_train = samples

    x_train = x_train.to(device)
    y_train = y_train.to(device)

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = criterion(prediction, y_train)

    # cost로 H(x) 계산
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4}/{} Batch {}/{} Cost: {:6f}'.format(
        epoch, nb_epochs, batch_idx+1, len(dataloader), cost.item()
    ))

Epoch    0/5 Batch 1/3 Cost: 196.565613
Epoch    0/5 Batch 2/3 Cost: 67.147804
Epoch    0/5 Batch 3/3 Cost: 147.798904
Epoch    1/5 Batch 1/3 Cost: 286.899658
Epoch    1/5 Batch 2/3 Cost: 480.501282
Epoch    1/5 Batch 3/3 Cost: 834.715820
Epoch    2/5 Batch 1/3 Cost: 1068.082764
Epoch    2/5 Batch 2/3 Cost: 945.053589
Epoch    2/5 Batch 3/3 Cost: 1670.123291
Epoch    3/5 Batch 1/3 Cost: 18.280916
Epoch    3/5 Batch 2/3 Cost: 23.409044
Epoch    3/5 Batch 3/3 Cost: 67.413734
Epoch    4/5 Batch 1/3 Cost: 68.401115
Epoch    4/5 Batch 2/3 Cost: 65.779694
Epoch    4/5 Batch 3/3 Cost: 116.002853
Epoch    5/5 Batch 1/3 Cost: 320.265900
Epoch    5/5 Batch 2/3 Cost: 619.203003
Epoch    5/5 Batch 3/3 Cost: 143.769745


In [30]:
#
#
#
#
#

In [3]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

In [4]:
learning_rate = 0.001 # 옵티마이저의 learning rate
training_epochs = 15 # 학습 횟수
batch_size = 100 # minibatch 사이즈, 전체 데이터셋에서 서브 셋을 몇개로 묶을거냐

In [5]:
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                          train=False,
                          transform=transforms.ToTensor(),
                          download=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 113368412.39it/s]


Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 18862611.93it/s]

Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 23302384.51it/s]


Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 7891685.49it/s]


Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



In [6]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                         batch_size=batch_size,
                                         shuffle=True,
                                         drop_last=True) # drop_last 마지막 남은 데이터를 버릴지 선택

In [4]:
# torch.nn.Module:  PyTorch의 모든 Neural Network의 Base Class
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # 첫번째층
        # ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 두번째층
        # ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 전결합층 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)

        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        out = self.fc(out)
        return out

In [8]:
# CNN 모델 정의
model = CNN().to(device)

In [9]:
criterion = torch.nn.CrossEntropyLoss().to(device)    # 비용 함수에 소프트맥스 함수 포함되어져 있음.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 600


In [11]:
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y는 레이블.
        # image is already size of (28x28), no reshape

        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.225602373
[Epoch:    2] cost = 0.0631315187
[Epoch:    3] cost = 0.0463035554
[Epoch:    4] cost = 0.0374769792
[Epoch:    5] cost = 0.0313607678
[Epoch:    6] cost = 0.0262179915
[Epoch:    7] cost = 0.0219778605
[Epoch:    8] cost = 0.0184681527
[Epoch:    9] cost = 0.0161481686
[Epoch:   10] cost = 0.0131634325
[Epoch:   11] cost = 0.00995935686
[Epoch:   12] cost = 0.0101486538
[Epoch:   13] cost = 0.00830312911
[Epoch:   14] cost = 0.00703515206
[Epoch:   15] cost = 0.00613190467


In [ ]:
from sklearn.metrics import f1_score

# 학습을 진행하지 않을 것이므로 torch.no_grad(), gradient descent를 하지마라고 명령내리는 것
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
                        # CNN은 10개의 아웃풋으로 각 10개의 클래스에 대한 피처값이 나온다
                        # 이를 axis 1방향으로 max값을 찾는다는 것
    correct_prediction = torch.argmax(prediction, 1) == Y_test
                          # torch.argmax(prediction, 1)은 피쳐값중 max인 인덱스를 반환
    
    print( f1_score(torch.argmax(prediction, 1), Y_test) )

    # accuracy = correct_prediction.float().mean()
    # print('Accuracy:', accuracy.item())

In [14]:
# 모델 저장
torch.save(model.state_dict(), "cnn_model.pt")

In [8]:
# 다시 불러와서 추론해보기
model = CNN().to(device)
model.load_state_dict(torch.load("cnn_model.pt"))
model.eval() # 평가 모드로 설정하여야 합니다. 이 과정을 거치지 않으면 일관성 없는 추론 결과가 출력

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=3136, out_features=10, bias=True)
)

In [14]:
# 학습을 진행하지 않을 것이므로 torch.no_grad(), gradient descent를 하지마라고 명령내리는 것

from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

with torch.no_grad():

    # 이미지 파일 경로 설정
    img = Image.open("./8.png")
    transform = transforms.Compose([
          transforms.Grayscale(num_output_channels=1), # RGB(3D) -> Gray(2D)
          transforms.Resize((28, 28)), # 모델 인풋에 맞게
          transforms.ToTensor(), # 토치 텐서 타입으로 맞춰줘야함
      ])
    
    img_tensor = transform(img).to(device) # [1, 28, 28]
    img_tensor = img_tensor.unsqueeze(0) # [1, 1, 28, 28] # 모델이 원래의 [배치사이즈, 채널, 가로, 세로]

    print(img_tensor.shape)

    prediction = model(img_tensor)
                        # CNN은 10개의 아웃풋으로 각 10개의 클래스에 대한 피처값이 나온다, 이를 axis 1방향으로 max값을 찾는다는 것 
    print('result:', torch.argmax(prediction, 1))

torch.Size([1, 1, 28, 28])
result: tensor([5], device='cuda:0')
